In [1]:
import pandas as pd
import numpy as np
import plotly.express

In [38]:
data = pd.read_excel('/content/ГПН-НТЦ.xlsx', sheet_name='Лист2')
data.head()

,WELL_ID,X_BHP,Y_BHP,h
0,110000204,451633,7595651,11
1,110001100,450897,7600603,9
2,110001200,452910,7601194,11
3,110001300,450897,7599984,10
4,110001400,453259,7600674,14


In [14]:
fig = plotly.express.scatter(data,  x='X_BHP', y='Y_BHP')
# fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species")

fig.update_traces(marker=dict(size=10,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.show()

In [4]:
def compute_S(x,y):
  """
  x, y: array or list-like objects: coordinates
  to do: Area calculation by coordinates
  return: Area"""
  resx, resy = 0, 0

  for i in range(len(x)-1):
    resx += x[i] * y[i+1]
  resx += x[-1] * y[0]

  for i in range(len(y)-1):
    resy += x[i+1] * y[i]
  resy += x[0] * y[-1]

  res = abs(resx - resy)/2

  return res

In [5]:
def match(arr):
  """
  arr: array or list-like object: array of arrays with [x,y] coordinates
  to do: Сonstruction of a minimal contour
  return: Array of indexes extreme points"""

  def rotate(A, B, C):
    """
    A, B, C: array or list-like object
    retutn: True (right) or False (left)
    """
    return ( B[0]-A[0] ) * ( C[1]-B[1] ) - ( B[1] - A[1] ) * ( C[0]-B[0] ) < 0

  n = len(arr)
  indxs = [i for i in range(n)]
  # finds left corner point
  for i in range(1,n):

    if arr[ indxs[i] ][0] < arr[ indxs[0] ][0]: 
      indxs[i], indxs[0] = indxs[0], indxs[i]  
  # make this point current
  points = [ indxs[0] ]
  del indxs[0]
  indxs.append(h[0])

  while True:

    r = 0
    for i in range( 1,len(indxs) ):

      if rotate( arr[points[-1]], arr[indxs[r]], arr[indxs[i]] ):
        r = i
    # breake while if current point is 
    if indxs[r]==h[0]: 
      break
    # else push point to list points
    else:
      points.append( indxs[r] )
      del indxs[r]
  
  return points   

In [6]:
A = np.array(data[ ['X_BHP', 'Y_BHP'] ])

In [36]:
d = [ A[i] for i in match(A) ]
plotly.express.scatter(d, x=[i[0] for i in d], y=[i[1] for i in d])

In [8]:
s = compute_S( x=[i[0] for i in d], y=[i[1] for i in d] )
print(f'Площадь равна {s} кв. метров')

Площадь равна 250911020.0 кв. метров


In [39]:
# плотность сетки
#Физический смысл плотности сетки скважин это площадь, 
#нефтенасыщенного пласта приходящаяся на одну эксплуатационную скважину
q = s/len(data.WELL_ID)
data['v_тыс._куб._м'] = [(q*i)/1000 for i in data.h]
# объемная плотность 
# qv = [s*i/len(data.WELL_ID)*i for i in data.h]
# data['v_тыс._куб._м'] = qv
data

,WELL_ID,X_BHP,Y_BHP,h,v_тыс._куб._м
0,110000204,451633,7595651,11,4654.335953
1,110001100,450897,7600603,9,3808.093052
2,110001200,452910,7601194,11,4654.335953
3,110001300,450897,7599984,10,4231.214503
4,110001400,453259,7600674,14,5923.700304
...,...,...,...,...,...
588,110805800,439907,7593380,12,5077.457403
589,110806200,440274,7596674,14,5923.700304
590,110806400,439752,7593334,14,5923.700304
591,110807000,439301,7593185,12,5077.457403
